https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f

https://exceptionshub.com/selenium-cant-click-element-because-other-element-obscures-it.html

https://www.toolsqa.com/selenium-webdriver/handle-iframes-in-selenium/

https://www.selenium.dev/documentation/webdriver/elements/finders/

https://www.selenium.dev/selenium/docs/api/javascript/module/selenium-webdriver/index_exports_By.html

In [1]:
import time, datetime, requests,random, os, hashlib

from pathlib import Path

from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
#//*** install xlsxwriter for pandas
#pip install xlsxwriter

#//*********************************************************
#//*** Build the tgt_date from the slected Quarter and Year
#//*********************************************************
tgt_year = 2022
quarter = "Q1"

if quarter == "Q1":
    tgt_date = f"{tgt_year}-01-01"
elif quarter == "Q2": 
    tgt_date = f"{tgt_year}-04-01"
elif quarter == "Q3": 
    tgt_date = f"{tgt_year}-07-01"
elif quarter == "Q4": 
    tgt_date = f"{tgt_year}-10-01"

tgt_date = datetime.datetime.strptime(tgt_date, "%Y-%m-%d")

print(tgt_date)

#//*** Get Filename for dataframe Cache
cache_filepath =  f"{tgt_year}_{quarter}_scripts.dat"
current_dir = Path(os.getcwd()).absolute()
cache_filepath = current_dir.joinpath(cache_filepath)

#//*** If Cache File Exists Load it. Else Start a Fresh DataFrame
if os.path.exists(cache_filepath):
    cache_df = pd.read_pickle(cache_filepath)
else:
    cache_df = pd.DataFrame()
                



2022-01-01 00:00:00


In [2]:
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for i,url in enumerate(source_urls):
        
        print(f"({i+1}/{len(source_urls)}) - {url}")
        try:
            page_driver.get(url)
        except:
            print("Problem Getting Page...Skipping")
            continue
            
        
        try:
        
            loop_story = {
                "url" : url,
                "headline" : page_driver.find_elements(By.CLASS_NAME,"headline")[0].text,
                "time_text" : page_driver.find_elements(By.CLASS_NAME,"lastmodified")[0].text,
                "body_text" : page_driver.find_elements(By.CLASS_NAME,"body-text")[0].text,
            }
            #//*** Split the Date
            loop_story['time_date'] = loop_story['time_text'].split()

            #//*** Rebuild Date string using only the first four fields. Drop the extra time
            loop_story['time_date'] = " ".join(loop_story['time_date'][:4])

            loop_story['time_date'] = datetime.datetime.strptime(loop_story['time_date'], "%A, %B %d, %Y")
        except:
            print("Problem Processing Story. Skipping")
            continue
        out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out


In [3]:
    
    
def get_all_urls_in_page(source_url,tgt_date,headless=True):

    #//*** Keep all classnames in a dictionary
    g = {
        #//*** Classname for the Show More Button
        "class_more_button" : "show-button-more",
        
        #//*** Grid Classname that contains the linked stories
        "story_grid" : "grid3",
        
        #//*** Story Classes that contain the href Links
        "story_link_class" : "AnchorLink",
        
    }
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        driver = webdriver.Firefox()


    # Load Web Page
    driver.get(source_url)
    
    # execute script to scroll down the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")


    #//*****************************************************
    #//*** Wait until the Show More Button is loaded
    #//*****************************************************
    elements = []
    control = 0
    while len(elements) == 0:

        #//*** Page is Loaded when class_more_button is displayed
        #elements = driver.find_elements_by_class_name(g["class_more_button"])
        elements = driver.find_elements(By.CLASS_NAME,g["class_more_button"])

        control += 1
        #//*** Wait if element not found
        if len(elements) == 0:
            time.sleep(1)

        if control > 15:
            print("Show More Button Not Loaded!")
            return
            break

    #//*************************************************
    #//*** The first elements is the button to click
    #//*************************************************
    elem_show_more = elements[0]

    #actions = ActionChains(driver)
    #actions.move_to_element(element).perform()

    print(elem_show_more.is_displayed())
    print(elem_show_more.location['y'])
    
    #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])
    results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])

    if len(results) > 0:
        print("Last Page: ", results[0].get_property('href'))

    
    
    #print(results[-1].get_property('href'))

    
    #print(story_time,tgt_date, story_time < tgt_date)
                                                                        
    #//*** Get the last url to check in stories are within the quarter
    last_url = results[-1].get_property('href')
    
    #//*** Download the last script
    last_script = get_scripts([last_url])[0]
    last_date = last_script['time_date']
    print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
    print("Last Script: ", last_script['time_date'] < tgt_date)
    
    #//*** Maximum number of while loops.
    maxDepth = 20
    depth = 0
    
    #//*** Click Show More until Last Story is older than the tgt_date
    while last_date > tgt_date:
        depth += 1
        
        #//*** Scroll to Bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        print(f"Clicking More Depth: {depth}")
        #print(dir(elem_show_more))
        while elem_show_more.is_displayed() == False:
            print("Is Displayed:",str(elem_show_more.is_displayed()))
            time.sleep(1)
        
        
        try:
            #//*** Click Show More
            elem_show_more.click()
        except:
            iframes = driver.find_elements_by_tag_name("iframe")
            for iframe in iframes:
                print(iframe.get_property("name"))
                
                
                driver.execute_script("arguments[0].style.visibility='hidden'", iframe)
            #//*** Click Show More
            elem_show_more.click()
                
        #    return

    
        #//*** Get Updated Results
        #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])        
        results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])        
        
        #//*** Get the last url to check in stories are within the quarter
        last_url = results[-1].get_property('href')

        #//*** Download the last script
        last_script = get_scripts([last_url])[0]
        last_date = last_script['time_date']
        print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
        print("Last Script: ", last_script['time_date'] < tgt_date)

        if depth > maxDepth:
            print("Maximum Depth Reached on Clicking Show More")
            break
        
    
    out = []
    
    for result in results:
        out.append(result.get_property('href'))
    
    #//*** Shut down main scrape
    driver.quit()
    
    

    return out




In [4]:
#//*** Hash url list and compare to cache_df
#//*** returns a tuple of urls (orig_urls,dupe_urls)
def keep_original_urls(urls,cache_df):
    orig_urls = []
    dupe_urls = []
    
    tdf = pd.DataFrame()
    tdf['urls'] = urls
    
    tdf['hash'] = pd.util.hash_pandas_object(tdf['urls'])


    #//*** Check for Duplicates
    tdf['dupe'] = tdf['hash'].isin(cache_df['hash'])

    #//*** Keep Only False Stories, which are not duplicates
    #//*** This means a story can have only a single category.
    #//*** May have to handle multiple categories at a future time
    #print(f"Length Before: {len(tdf)} ")

    orig_urls = list(tdf[tdf['dupe'] == False]['urls'])
    dupe_urls = list(tdf[tdf['dupe'] == True]['urls'])

    
    
    return orig_urls,dupe_urls

In [ ]:
url_items = [
    {
        "url" : "https://abc7news.com/tag/climate-change/",
        "cat" : "Climate"
    },
    {
        "url" : "https://abc7news.com/tag/environment/",
        "cat" : "Environment"
    },
 
    {
        "url" : "https://abc7news.com/tag/economy/",
        "cat" : "Economy"
    },
    {
        "url" : "https://abc7news.com/tag/jobs/",
        "cat" : "Jobs"
    },    
    {
        "url" : "https://abc7news.com/tag/safety/",
        "cat" : "Safety"
    },
    {
        "url" : "https://abc7news.com/tag/health/",
        "cat" : "Health"
    },
    {
        "url" : "https://abc7news.com/education/",
        "cat" : "Education"
    },
    {
        "url" : "https://abc7news.com/tag/race-and-culture/",
        "cat" : "Race"
    },
    {
        "url" : "https://abc7news.com/tag/discrimination/",
        "cat" : "Discrimination"
    },
    {
        "url" : "https://abc7news.com/tag/civil-rights/",
        "cat" : "Civil Rights"
    },
    {
        "url" : "https://abc7news.com/tag/building-a-better-bay-area/",
        "cat" : "BABBA"
    },
    {
        "url" : "https://abc7news.com/7onyourside/",
        "cat" : "7OYS"
    },
    {
        "url" : "https://abc7news.com/tag/covid-19/",
        "cat" : "COVID"
    },

    {
        "url" : "https://abc7news.com/iteam/",
        "cat" : "I-TEAM"
    },
    

    
]
urlpages = ['https://abc7news.com/education/',
            'https://abc7news.com/tag/building-a-better-bay-area/',
            'https://abc7news.com/7onyourside/',
            'https://abc7news.com/tag/covid-19/',
            'https://abc7news.com/tag/climate-change/',
            'https://abc7news.com/iteam/'
           ]

#urlpage ='https://abc7news.com/education/'
#urls = get_all_urls_in_page(urlpages[1],tgt_date,False)
#print(urlpage)

#//*** Loop through top level url_items
for item in url_items:
    cat_url = item['url']
    cat = item['cat']
    
    #//*** Download the ctageory URLs
    urls = get_all_urls_in_page(cat_url,tgt_date,False)
    
    #//*** Get non-duplicate urls to download and a list of the duplicate urls
    #//*** We may need to incorporate duplicates with different categories...later
    if len(cache_df) > 0:
        urls_to_dl,dupe_urls = keep_original_urls(urls,cache_df)
    else:
        #//*** There is no cache initialize and move one
        urls_to_dl = urls
        dupe_urls = []
    print("=====================")
    print(f"Category: {cat}")
    print("=====================")
    print(f"DL Items: {len(urls_to_dl)} - Duplicate: {len(dupe_urls)}")
    
    if len(urls_to_dl) == 0:
        print("No New Items to Download!")
    
    if len(urls_to_dl) > 0:
        scripts = get_scripts(urls_to_dl)
        
        #//*** Convert Scripts to DataFrame
        df = pd.DataFrame()
        #df['urls'] = urls_to_dl

        s = {}

        #//*** Get each script value as a list
        for script in scripts:

            for key,value in script.items():
                if key not in s.keys():
                    s[key] = []
                s[key].append(value)

        #//*** build Urls field
        #//*** Use the Story URL in case an url was skipped.
        df['urls'] = s['url']
        df['hash'] = pd.util.hash_pandas_object(df['urls'])
        df['cat'] = cat

        #//*** Add lists of script values as columns
        for key,value in s.items():
            df[key] = value

        #//*** Combine cache_df and df
        cache_df = pd.concat([cache_df,df])


#//*** Everything is Gathered. Write Cache_df to disk
#//*** Write DF to file
pd.to_pickle(cache_df,cache_filepath)
    

True
1805
Last Page:  https://abc7news.com/rain-california-drought-2022-san-jose-weather-western/11571287/
(1/1) - https://abc7news.com/sf-crab-season-delayed-fishermans-wharf-crabs/11485170/
Last Script:  San Francisco fishermen struggle to stay afloat amid delayed crab season 
 Tuesday, January 18, 2022
Last Script:  False
Clicking More Depth: 1
(1/1) - https://abc7news.com/cca-california-college-of-the-arts-architecture-climate-change/11344973/
Last Script:  'Idea of hope': SF college students design sustainable architecture that helps ecosystems flourish 
 Wednesday, December 15, 2021
Last Script:  True
Category: Climate
DL Items: 24 - Duplicate: 0
(1/24) - https://abc7news.com/rain-california-drought-2022-san-jose-weather-western/11571287/
(2/24) - https://abc7news.com/stanford-double-hazard-wildfire-west-zones-california-sierra-nevada/11568793/
(3/24) - https://abc7news.com/sea-level-rise-scientists-warn-climate-change-coastal-floods/11568385/
(4/24) - https://abc7news.com/levee-

(10/36) - https://abc7news.com/san-francisco-oakland-jose-bay-area-rent/11571718/
(11/36) - https://abc7news.com/truck-emissions-california-air-resources-board-supply-chain-issues-semi-trucks-engine-2010-or-new-models/11559969/
(12/36) - https://abc7news.com/bay-area-freeway-tolls-pay-california-traffic-metropolitan-transportation-commission/11556669/
(13/36) - https://abc7news.com/inflation-rate-report-cpi-us/11551708/
(14/36) - https://abc7news.com/grocery-prices-food-stores-costs/11548558/
(15/36) - https://abc7news.com/canada-covid-19-covid-19-protests-us-trade-economy/11546745/
(16/36) - https://abc7news.com/sf-new-amazon-delivery-hub-facility-mayor-agreement-labor-union-workers/11530654/
(17/36) - https://abc7news.com/rams-vs-49ers-sf-watch-party-in-bars-to/11521794/
(18/36) - https://abc7news.com/meat-prices-supply-chain-shortages-empty-shelves-at-stores/11509193/
(19/36) - https://abc7news.com/appraisal-home-appraisals-industry-reform/11489431/
(20/36) - https://abc7news.com/cl

(23/24) - https://abc7news.com/new-years-eve-great-america-californias-americas-winter-fest-santa-clara/11413117/
(24/24) - https://abc7news.com/san-jose-center-for-the-performing-arts-new-years-eve-events-nye-christmas-in-park/11409237/
True
1808
Last Page:  https://abc7news.com/top-tax-questions-free-help-when-will-i-get-my-w2-file-as-dependent-or-independent/11609457/
(1/1) - https://abc7news.com/noncitizen-voting-san-francisco-recall-election-rights-jose/11556464/
Last Script:  Non-citizen voting rights gain traction as immigrants vote in SF Unified school board recall 
 Friday, February 11, 2022
Last Script:  False
Clicking More Depth: 1
(1/1) - https://abc7news.com/vidsig-video-chat-college-applications-answers/11506642/
Last Script:  Bay Area-based company allows students to video-chat students at their potential colleges 
 Monday, January 24, 2022
Last Script:  False
Clicking More Depth: 2
(1/1) - https://abc7news.com/palo-alto-unified-pausd-pasud-covid-testing-schools/11452151

(10/24) - https://abc7news.com/love-a-state-of-grace-cathedral-exhibit-new-sf-art-joanna-haigood/11552545/
(11/24) - https://abc7news.com/alameda-gym-ad-post-island-personal-training-and-massage-online-against-asian-americans/11538787/
(12/24) - https://abc7news.com/kingmakers-of-oakland-improving-education-for-black-boys-history-month-nonprofits-to-help-community/11537906/
(13/24) - https://abc7news.com/black-history-month-domini-hoskins-museum-redwood-city-mlk/11536277/
(14/24) - https://abc7news.com/black-mental-health-african-american-community-service-agency-history-month-san-jose-resources/11530854/
(15/24) - https://abc7news.com/janet-jackson-girls-leadership-oakland-nonprofit/11533548/
(16/24) - https://abc7news.com/lunar-new-year-of-the-tiger-asian-tradition-asian-american-traditions/11530599/
(17/24) - https://abc7news.com/asian-senior-crime-sf-attacks-assaults-americans/11527334/
(18/24) - https://abc7news.com/into-the-depths-podcast-natgeo-undiscovered-slave-shipwrecks-abou

Category: BABBA
DL Items: 144 - Duplicate: 0
(1/144) - https://abc7news.com/how-much-is-gas-in-california-prices-near-me-bay-area-shortage-2022/11618233/
(2/144) - https://abc7news.com/san-francisco-homeless-problem-population-sf/11616180/
(3/144) - https://abc7news.com/seawater-desalination-huntington-beach-california-drought-water-shortages/11615559/
(4/144) - https://abc7news.com/san-francisco-financial-district-coronavirus-sf-return-to-work/11615591/
(5/144) - https://abc7news.com/ca-school-mask-mandate-sfusd-masks-mayor-london-breed-lifting/11612409/
(6/144) - https://abc7news.com/methane-emissions-pge-greenhouse-gas-merced/11587342/
(7/144) - https://abc7news.com/san-francisco-black-firefighter-sf-earl-gage-jr-rosa-parks-elementary-mural-first/11600497/
(8/144) - https://abc7news.com/the-patient-story-cancer-questions-support-stephanie-chuang/11599948/
(9/144) - https://abc7news.com/california-reparations-bill-task-force-ca-vote/11597465/
(10/144) - https://abc7news.com/drug-over

(79/144) - https://abc7news.com/coronavirus-and-fertility-rate-after-covid-mens-health-men/11506177/
(80/144) - https://abc7news.com/oakland-schools-strike-ousd-educators-association-oea/11503445/
(81/144) - https://abc7news.com/omicron-peak-fauci-bay-area-cases-february/11503457/
(82/144) - https://abc7news.com/colorado-fire-big-sur-monterey-county-california-wildfires/11500438/
(83/144) - https://abc7news.com/restaurant-visits-drop-in-california-bay-area-restaurants-foot-traffic-drops-iteam-visit/11496772/
(84/144) - https://abc7news.com/sccfreetest-santa-clara-county-covid-cases-free-tests-usps-kits/11496472/
(85/144) - https://abc7news.com/appraisal-home-appraisals-industry-reform/11489431/
(86/144) - https://abc7news.com/bart-bay-area-rapid-transit-independent-institute-schedule/11488578/
(87/144) - https://abc7news.com/usps-free-covid-tests-covidtest-gov-test-website-covidtests/11485072/
(88/144) - https://abc7news.com/covid-cases-california-us-usps-free-tests-test-gov/11485381/


(9/35) - https://abc7news.com/sf-bottlebank-mobile-recycling-service-san-francisco-app/11538522/
(10/35) - https://abc7news.com/bank-of-america-bofa-loses-check-oakland-man-33000-dollars/11536348/
(11/35) - https://abc7news.com/california-unemployment-edd-fraud-disability-claims/11533538/
(12/35) - https://abc7news.com/california-unemployment-edd-fraud-disability-claims/11533538/
(13/35) - https://abc7news.com/bank-of-america-bofa-loses-check-deposit-lost-transfer-money/11530499/
(14/35) - https://abc7news.com/7-on-your-side-7oys-michael-finney-consumer/11517850/
(15/35) - https://abc7news.com/7-on-your-side-7oys-michael-finney-consumer/11515441/
(16/35) - https://abc7news.com/edd-disability-claims-fraud-locked-account-frozen/11514940/
(17/35) - https://abc7news.com/consumer-alert-sugar-baby-scam-teenage-curiosity-zelle/11508213/
(18/35) - https://abc7news.com/edd-disability-claims-fraud-locked-account-frozen/11506004/
(19/35) - https://abc7news.com/credit-scores-improve-score-boost-fi

Problem Processing Story. Skipping
(33/108) - https://abc7news.com/7-on-your-side-7oys-michael-finney-consumer/11515441/
(34/108) - https://abc7news.com/omicron-subvariant-ba2-bay-area-2-cases-santa-clara-county/11515126/
(35/108) - https://abc7news.com/san-mateo-ppe-waste-department-of-homeland-security-investigation-fraud/11515977/
(36/108) - https://abc7news.com/heart-transplant-denied-covid-19-vaccine-boston-hospital-policy/11509056/
(37/108) - https://abc7news.com/san-mateo-county-ppe-waste-personal-protection-equipment-icu-eyewear/11509605/
(38/108) - https://abc7news.com/neil-young-music-off-spotify-covid-19-vaccine-misinformation/11508141/
(39/108) - https://abc7news.com/elton-john-covid-farewell-yellow-brick-road-dallas-postponed-houston-show/11508093/
(40/108) - https://abc7news.com/n95-mask-free-how-do-you-get-walgreens-cvs-near-me/11507892/
(41/108) - https://abc7news.com/california-vaccine-mandate-students-covid/11505335/
(42/108) - https://abc7news.com/crystal-cruises-arr

In [ ]:
#//*** Write to excel file
output_filename = f"{tgt_year}_{quarter}_Collected_Stories.xlsx"
print(output_filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

#//*** Build Sheet and Data from From each url_item element
for url_item in url_items:
    sheet = url_item['cat']
    
    tdf = cache_df[cache_df['cat'] == sheet].copy()
    
    tdf = tdf[['urls','time_text','headline','body_text']]
    
    tdf.to_excel(writer,sheet_name=sheet)
    
try:
    writer.save()    
except:
    print("Trouble Saving the Spreadsheet. It's Probably Open Somewhere. Close it and Retry")    
    